A notebook for training a music generation transformer

In [1]:
import sys
sys.path.insert(0, '../src/libs')

In [2]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [3]:
#midi_path = '../src/mid_data_collections/mid_0_to_10000'
numpy_path = '../src/numpy_path'

In [4]:
numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)
#midi_files = get_files(midi_path, '.mid', recurse=True)
#print(midi_files[0])
#print(len(midi_files))

1177

In [5]:
data_path = './saved_data/'
data_save_name = 'music_item_data_0-10000.pkl'
def create_databunch(files, path):
    #save_file.parent.mkdir(exist_ok=True, parents=True)
    vocab = MusicVocab.create()
    processors = [OpenNPFileProcessor(), MusicItemProcessor()]

    #data = MusicDataBunch.from_files(midi_files, path, processors=processors, encode_position=True)
    data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
    return data

In [6]:
all_data = create_databunch(numpy_files, data_path); all_data

C:\Users\Trevi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\fastai\core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


MusicDataBunch;

Train: LabelList (1060 items)
x: MusicItemList

MusicItem - (9818,)
xxbos xxpad n54 d2 n50 d2 n38 d1 xxsep d1...,
MusicItem - (6616,)
xxbos xxpad n64 d2 n61 d2 n57 d2 n52 d4...,
MusicItem - (9758,)
xxbos xxpad xxsep d7 n54 d11 xxsep d1 n64 d11...,
MusicItem - (1236,)
xxbos xxpad n76 d32 n72 d32 n67 d32 n64 d32...,
MusicItem - (7064,)
xxbos xxpad n76 d2 n72 d2 n60 d29 n57 d28...
y: LMLabelList
,,,,
Path: saved_data;

Valid: LabelList (117 items)
x: MusicItemList

MusicItem - (11442,)
xxbos xxpad xxsep d9 n55 d1 xxsep d1 n57 d1...,
MusicItem - (15824,)
xxbos xxpad xxsep d11 n89 d1 n88 d1 xxsep d1...,
MusicItem - (2552,)
xxbos xxpad n70 d10 n67 d10 n62 d10 n58 d10...,
MusicItem - (5066,)
xxbos xxpad n58 d1 n55 d1 n51 d1 n39 d4...,
MusicItem - (5798,)
xxbos xxpad n60 d128 n55 d32 n51 d32 n48 d12...
y: LMLabelList
,,,,
Path: saved_data;

Test: None

In [ ]:
def run_epoch(data_iter, model, loss_compute):#, bad_idxs):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    i = 0
    collection_of_losses = []
    for batch in data_iter:
        #print("batch num: ", i)
      #if i in bad_idxs:
      #  print("BAD IDX: ", i)
      #  i += 1
      #  continue  
      #else:
        out, w2 = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)

        loss = loss_compute(out, batch.trg_y, batch.ntokens.item())

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 200 == 1:
            collection_of_losses.append(torch.div(loss,batch.ntokens).cpu())
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, torch.div(loss,batch.ntokens), tokens / elapsed))
            start = time.time()
            tokens = 0
        i += 1
    return (total_loss / total_tokens), collection_of_losses

In [ ]:
# My version of Greedy Decoding
def test_my_model():

  #---------- TRAIN -----------
  V = len(all_data.vocab) # 312
  criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
  model = make_model(V, V, N=2, d_model=128)
  model.cuda() # uncomment for GPU
  model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400, torch.optim.Adam(model.parameters(), lr=0.6, betas=(0.9, 0.98), eps=1e-9))
  the_train_data = data_gen(all_data.train_ds)
  collection_of_losses = np.array([])
  for epoch in range(30):
    print("epoch: ", epoch)
    model.train()
    totalLoss_div_totalTokens, collection_of_losses_inp = run_epoch(the_train_data, model, SimpleLossCompute(model.generator, criterion, model_opt))#, bad_idxs)
    #model.eval()
    #print(run_epoch(the_train_data, model, SimpleLossCompute(model.generator, criterion, None), bad_idxs))
    collection_of_losses = np.concatenate([collection_of_losses, collection_of_losses_inp])
  return model, collection_of_losses

model, losses = test_my_model()

Save model to a .pt file

In [ ]:
torch.save(model, 'epochs_75_LM.pt')

Load the Model from a previous state

In [ ]:
myModel = torch.load("./epochs_30_LM.pt")